In [22]:
from __future__ import print_function
from keras.models import Sequential
from keras.layers import *
from keras.layers import Input
from keras.optimizers import RMSprop
from keras.utils.data_utils import get_file
import keras.backend as K
import numpy as np
import tarfile
import itertools
from keras.models import *
import random
import sys
from glob import glob
import re
import tqdm
from keras.preprocessing.sequence import pad_sequences
from keras.utils.np_utils import *
import os
from keras_tqdm import TQDMNotebookCallback

# Corpus Preprocessing

### Data Format

The [training data](https://research.fb.com/downloads/babi/) consists of a number of "examples". Each example contains a few memories, and the relevant question/answer pairs. Consider the following example

1. Mary moved to the bathroom.
2. John went to the hallway.
3. Where is Mary?        bathroom        1
4. Daniel went back to the hallway.
5. Where is Daniel?     hallway  4

In the above example, we have 3 memories and 2 question/answer pairs:

- Three Memories
    * Mary moved to the bathroom.
    * John went to the hallway.
    * Daniel went back to the hallway.
    
- 2 Questions/Answer Pairs:
    * Q: *Where is Mary?* 
      A: bathroom
    
    * Q: *Where is Daniel?*
      A: hallway
      
 Note that for the first question, only memories (1, 2) are relevant. For the second question, memories (1, 2, 4) are relevant.
 

### Abstractions
* Corpus
: Responsible for keeping tracks of the information related to the text corpus. For example, the total number of unique
    words, mapping from string to text and so on.
* ExampleParser
: The training file consists of "example" units. ExampleParser takes a file with a number of examples and parses them into 3-tuples of (memories, question, answer)
    
    

In [23]:
class Corpus:
    def __init__(self):
        '''
        A corpus object maintains a mapping from a word (string) to a unique id (int).
        '''
        self.word_idx_dict = {}
        self.uniq_word_cnt = 0
    
    def update_vocab(self, words):
        '''
        Updates the corpus with the given list of words. The words that are seen for the 
        first time are added to the word -> id dictionary.
        '''
        for word in words:
            if word not in self.word_idx_dict:
                self.word_idx_dict[word] = self.uniq_word_cnt
                self.uniq_word_cnt += 1

    def words_idx(self, words):
        '''
        Returns the list of IDs corresponding to the given words. 
        '''
        return [self.word_idx_dict[word] for word in words]
    
    def tokenize(self, sentence):
        return [x.strip() for x in re.split('(\W+)?', sentence) if x.strip()]

In [24]:
class ExampleParser:
    '''
    Responsible for parsing examples for the babi tasks as specified at https://research.fb.com/downloads/babi/
    '''
    @staticmethod
    def add(example_lines, data, c):
        '''
        Takes the set of lines that form an example and:
        a) updates the corpus with these lines 
        b) Parses the line to 3-tuples of the form: (memory, question, answer).
        
        A single story line may yield several 3-tuples of the above form. E.g.: 
        1 Mary moved to the bathroom.
        2 John went to the hallway.
        3 Where is Mary?        bathroom        1
        4 Daniel went back to the hallway.
        12 Where is Daniel?     hallway  4
        
        Will generate 2 tuples: 
        
        - ([1 Mary moved to the bathroom., 2 John went to the hallway.], Where is Mary?, bathroom)
        - ([1 Mary moved to the bathroom., 2 John went to the hallway., 4 Daniel went back to the hallway.],
                                                                         Where is Daniel?, hallway)
        
        Note that instead of storing the actual words, an example stores the IDs of the associated words. A word -> ID
        map is maintained in the corpus.
        
        data: List of 3-tuples (memories, question, answer), updated by "add".
        c: The corpus object.
        '''
        memories = []
        memories_txt = []
        qa = []
        for eg_line in example_lines:
            if "\t" not in eg_line: #normal memory
                eg_line = c.tokenize(eg_line)
                c.update_vocab(eg_line)
                mem_id, memory = eg_line[0], c.words_idx(eg_line[1:])
                memories.append(c.words_idx(eg_line))
                memories_txt.append(eg_line)
            else: #question line
                ques, ans, hints = eg_line.split("\t")
                ques = c.tokenize(ques)[1:]
                c.update_vocab(ques)
                ans = c.tokenize(ans)
                c.update_vocab(ans)
                data.append(([m for m in memories],
                                  c.words_idx(ques), c.words_idx(ans)))
    
    @staticmethod
    def process_files(lines, corpus):
        '''
        Reads the given file, identifies splits of the example and adds them to th.
        '''
        data = []
        eg_lines = [lines[0].decode('utf-8').strip()]
        for line in lines[1:]:
            line = line.decode('utf-8').strip()
            if int(line.split(" ", 1)[0]) == 1: #new story starts
                ExampleParser.add(eg_lines, data, corpus)
                eg_lines = [line.strip()]
            else:
                eg_lines.append(line.strip())
        if len(eg_lines) > 0:
            ExampleParser.add(eg_lines, data, corpus)
        return data

### Download and extract datasets

In [25]:
challenges = {
    # QA1 with 10,000 samples
    'single_supporting_fact_10k': 'tasks_1-20_v1-2/en-10k/qa1_single-supporting-fact_{}.txt',
    # QA2 with 10,000 samples
    'two_supporting_facts_10k': 'tasks_1-20_v1-2/en-10k/qa2_two-supporting-facts_{}.txt',
}
path = get_file('babi-tasks-v1-2.tar.gz', origin='https://s3.amazonaws.com/text-datasets/babi_tasks_1-20_v1-2.tar.gz')
tar = tarfile.open(path)
challenge_type = 'single_supporting_fact_10k'
challenge = challenges[challenge_type]

test_lines = tar.extractfile(challenge.format('test')).readlines()
train_lines = tar.extractfile(challenge.format('train')).readlines()
print(train_lines[0:4])
print(test_lines[0:4])

[b'1 Mary moved to the bathroom.\n', b'2 John went to the hallway.\n', b'3 Where is Mary? \tbathroom\t1\n', b'4 Daniel went back to the hallway.\n']
[b'1 John travelled to the hallway.\n', b'2 Mary journeyed to the bathroom.\n', b'3 Where is John? \thallway\t1\n', b'4 Daniel went back to the bathroom.\n']


### Parse datasets to (memories, question, answer) tuples, perform word -> idx mapping

In [26]:
c = Corpus()
print("Processing training files")
train_tuples = ExampleParser.process_files(train_lines, c)
print("Processing test files")
test_tuples = ExampleParser.process_files(test_lines, c)
all_tuples = train_tuples + test_tuples
print("# training tuples = {0}\n# test tuples = {1}".format(len(train_tuples), len(test_tuples)))


Processing training files


C:\Users\amadaan\AppData\Local\Continuum\Anaconda3\lib\re.py:203: FutureWarning: split() requires a non-empty pattern match.
  return _compile(pattern, flags).split(string, maxsplit)


Processing test files
# training tuples = 10000
# test tuples = 1000


#### Example Description
- Let us look at the first example and the corresponding lines from the text.

In [27]:
print(train_tuples[0])
print(train_lines[0:3])

([[0, 1, 2, 3, 4, 5, 6], [7, 8, 9, 3, 4, 10, 6]], [11, 12, 1, 13], [5])
[b'1 Mary moved to the bathroom.\n', b'2 John went to the hallway.\n', b'3 Where is Mary? \tbathroom\t1\n']


* From the mapping, note that "1 Mary moved to the bathroom." has been mapped to "0, 1, 2, 3, 4, 5, 6". (line number is used as a feature).
    - 1 -> 0
    - Mary -> 1
    - moved -> 2
    - to -> 3
    - the -> 4
    - bathroom -> 5
    - . (full stop) - 6

* Similarily, 3 Where is Mary? is mapped to [11, 12, 1, 13] (note that Mary -> 1, the mapping is retained).

* The answer is 5, bathroom

## Preparing Dataset for Training

- At this point, we have the 3-tuples of (memories, question, answer). 

- Different tuples may have different number of memories, and different number of words in memories and questions.

- As a reminder, a 3-tuple contains a bunch of memories, 1 question and the corresponding answer. Also, we have processed these tuples so that at this point, they are just a bunch of numbers.

- Because we want to use the same network for training, we will "pad" our 3-tuples to make them all of the same size. This means that:

1. All the 3-tuples should have the same number of memories.
2. All the memories should have the same number of words.
3. All the questions should have the same number of words.


- To achieve this, we will 0 pad the sequences. Let's find out relevant upper limits.

In [28]:
max_num_memories = max([len(example[0]) for example in all_tuples])
max_memory_len = max([len(memory) for example in all_tuples for memory in example[0]])
max_ques_len = max([len(example[1]) for example in all_tuples])
vocab_size = c.uniq_word_cnt
len(train_tuples), len(test_tuples), c.uniq_word_cnt, max_num_memories, max_memory_len, max_ques_len

(10000, 1000, 31, 10, 8, 4)

| Variable | Count |
|---|---|
| # Training Tuples | 10000 |
| # Test Tuples | 1000 |
| Max # words per memory | 8 |
| Max # memories in a tuple | 10 |
| Max # words per question | 4 |

In [29]:
def get_dataset_from_examples(examples, max_memory_len, max_num_memories, max_ques_len, vocab_size):
    '''
    Takes a number of examples, as well as measures required for 0 padding. 
    Returns a padded version of the memories, questions and the answer.
    In other words, for each tuple, memories is now a matrix of:
    a) max_num_memories * max_memory_len
    b) question is an array with max_ques_len elements. The gaps are filled with 0s.
    Also performs 1-hot encoding for the output.
    '''
    m, q, a = [], [], []
    for (memories, ques, ans) in examples:
        memories= pad_sequences(memories, maxlen=max_memory_len)
        memories = np.concatenate([memories, np.zeros((max_num_memories - memories.shape[0],
                                                       max_memory_len), 'int') ])
        m.append(memories)
        q.append(ques)
        #ans_vec = np.zeros((vocab_size))
        #ans_vec[ans] = 1
        a.append(ans)
    return np.array(m), pad_sequences(q, maxlen=max_ques_len), np.array(a)

In [30]:

m_train, q_train, a_train = get_dataset_from_examples(train_tuples,
                                                     max_memory_len,
                                                     max_num_memories,
                                                     max_ques_len,
                                                     vocab_size=c.uniq_word_cnt)

m_test, q_test, a_test = get_dataset_from_examples(test_tuples,
                                                     max_memory_len,
                                                     max_num_memories,
                                                     max_ques_len,
                                                     vocab_size=c.uniq_word_cnt)

print(m_train.shape) 
print(q_train.shape)
print(a_train.shape)
print(m_test.shape)
print(q_test.shape)
print(a_test.shape)

(10000, 10, 8)
(10000, 4)
(10000, 1)
(1000, 10, 8)
(1000, 4)
(1000, 1)


## Model

We will now build the model as specified at https://arxiv.org/abs/1503.08895.


The relevant snippet from the paper are referenced inline. An attempt has been made to stay as close to the notation in paper as possible. Continuing from the notations used above, our network is designed to be trained on 1-tuple at a time. That is, the network will be fed a list of 3-tuples, where each of the 3-tuples is (memories, question, answer). 

For the entirety of the model building discussion, the focus will be on 1 such tuple. 

### Step 1: Encode the inputs

The set x1, x2, ..., x_i that the paper mentions is our list of memories (each of the x_i is basically a list of numbers).

![alt text](snippets\m_i.png "Input Encoding")

### m_i
* For each of the inputs, x_i we learn an embedded representation, m_i. In the single fact example, the length of each x is 8. Each of the x is thus an 8-dimensional vectory. Similarily, the number of dimensions in the embedded space is 64. Thus, we want to map each of the 8 dim input vector to a 64 dim vector.

* This is achieved by embedding each word of the input to a 64 dim vector, and then adding them all to give the 64 dim vector for the input.

### Mapping m_1 to x_1: A complete example.

* Suppose that x_1 is = "1 Mary went to the kitchen", mapped to [0 0 12 23 32 33 22 21] (0 padding to make the length = 8).
 For brevity, suppose that the number of hidden dimensions was **5** (it's 64 in the real case), then step 1 would be to learn 1 5 dimensional vector for each word in the sequence:

- 0: '[0.13 0.69 0.52 -0.22 -1.15]'
- 0: '[0.13 0.69 0.52 -0.22 -1.15]'
- 1: '[0.05 0.23 -1.09 0.9 0.21]'
- Mary: '[0.8 -1.09 -0.49 1.7 2.77]'
- went: '[-0.95 0.87 0.88 0.02 0.09]'
- to: '[0.73 -0.58 0.27 0.95 0.9]'
- the: '[-1.13 0.41 -0.19 -1.41 0.07]'
- kitchen: '[-0.5 , -1.11, -0.47,  0.09, -1.87]'

 The representation for the sentence is then just a sum of all of these: [-0.77, -0.18, -1.46,  3.19, -1.]).
 
 This is m_1.

In [40]:
n_hidden = 64
memories = Input(shape=(max_num_memories, max_memory_len))
x = TimeDistributed(Embedding(input_dim=vocab_size, output_dim=n_hidden))(memories)
m = Lambda(lambda xx: K.sum(xx, 2))(x)
memories.shape, m.shape

(TensorShape([Dimension(None), Dimension(10), Dimension(8)]),
 TensorShape([Dimension(None), Dimension(10), Dimension(64)]))

## Step 2: Encode the query

![alt text](snippets\u.png "Query Encoding")

* Similarily, a query input is mapped from a 4 dim to a 64 dim space. This embedded representation of the query is called "u".

In [41]:
query = Input(shape=(max_ques_len,))
u = Embedding(input_dim=vocab_size, output_dim=n_hidden)(query)
u = Lambda(lambda x : K.sum(x, 1))(u)
u = Reshape(target_shape=(1, n_hidden))(u)
query.shape, u.shape

(TensorShape([Dimension(None), Dimension(4)]),
 TensorShape([Dimension(None), Dimension(1), Dimension(64)]))

## Step 3: Calculate p, the attention that each input deserves for the given query

* Our answers need to be derived from our memories. Further, similarities in the query and memory is a good indicator of the memory containing the answer (because of presence of common words). The similarity is calculated by taking a dot product of the embedded query with each of the embedded input, followed by a softmax.

![alt text](snippets\p.png "P")


In [42]:
p = dot([m, u], axes=2)
p = Reshape((max_num_memories,))(p)
p = Activation(activation='softmax')(p)
p = Reshape((max_num_memories,1))(p)
p.shape


TensorShape([Dimension(None), Dimension(10), Dimension(1)])

## Step 4: Calculate c, the output embeddings

* The input memory embeddings hopefully now have values that are conducive to finding out the relevant memories for a given question. We now want to learn another representation that will help in finding out the answer. The mechanics of finding out this embedding remain similar to what we did for m_i.

![alt text](snippets\c_i.png "Output Representation")

In [43]:
x = TimeDistributed(Embedding(vocab_size, n_hidden))(memories)
c = Lambda(lambda xx: K.sum(xx, 2))(x)
c.shape

TensorShape([Dimension(None), Dimension(10), Dimension(64)])

## Step 5: Calculate o, the output representation

* The output representation is calculated by taking a "weighted average" of the output memory repreesntation. Where the weights are given by p learned above.

## Output Representation, o
![alt text](snippets/o.PNG "Output Representation")

In [44]:
o = dot([c, p], axes=1)
o = Reshape(target_shape=(1,n_hidden))(o)
o

<tf.Tensor 'reshape_14/Reshape:0' shape=(?, 1, 64) dtype=float32>

## Step 6: Feed o + u to a softmax to get the answer
![](snippets/a.png "Answer")

In [45]:
a_in = Lambda(lambda ou: sum([ou[0], ou[1]]))([o, u])
a_in = Reshape(target_shape=(n_hidden,))(a_in)
answer = Dense(vocab_size, activation='softmax')(a_in)
answer

<tf.Tensor 'dense_3/Softmax:0' shape=(?, 31) dtype=float32>

In [46]:
babi_memmn = Model([memories, query], answer)

In [47]:
babi_memmn.compile(optimizer='rmsprop', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [48]:
K.set_value(babi_memmn.optimizer.lr, 1e-2)
params = {'verbose': 2, 'callbacks': [TQDMNotebookCallback(leave_inner=False)]}
babi_memmn.fit([m_train, q_train], a_train, **params, batch_size=32, epochs=5,
               validation_data=([m_test, q_test], a_test))

Train on 10000 samples, validate on 1000 samples
Epoch 1/5
1s - loss: 0.2716 - acc: 0.9073 - val_loss: 0.0022 - val_acc: 1.0000
Epoch 2/5
1s - loss: 0.0341 - acc: 0.9909 - val_loss: 0.0570 - val_acc: 0.9870
Epoch 3/5
1s - loss: 0.0338 - acc: 0.9924 - val_loss: 0.0015 - val_acc: 0.9990
Epoch 4/5
1s - loss: 0.0187 - acc: 0.9956 - val_loss: 0.0310 - val_acc: 0.9940
Epoch 5/5
1s - loss: 0.0224 - acc: 0.9958 - val_loss: 0.0034 - val_acc: 0.9990

